# Práctica 4: Procesamiento del Lenguaje Natural

__Fecha de entrega: 14 de mayo de 2024__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de PLN.

Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.

Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

# 1) Carga del conjunto de datos

El fichero `spam.csv` contiene mensajes SMS etiquetados como spam or ham (legítimo).

Muestra un ejemplo de cada clase.

Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?, ¿tendría sentido normalizar de alguna manera el corpus?

Crea una partición de los datos dejando el 60% para entrenamiento, 20% para validación y el 20% restante para test. Comprueba que la distribución de los ejemplos en las particiones es similar.

In [ ]:
RANDOM_STATE = 1234

In [ ]:
# acceso a google drive
from google.colab import drive
drive.mount('/content/drive')


In [47]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
df1 = pd.read_csv('/content/drive/MyDrive/spam.csv')
df1

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5568,ham,Will �_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN
5571,ham,Rofl. Its true to its name,NaN,NaN,NaN


 PO Box 5249
 the person is definitely special for u..... But if the person is so special
 HOWU DOIN? FOUNDURSELF A JOBYET SAUSAGE?LOVE JEN XXX\""
 wanted to say hi. HI!!!\" Stop? Send STOP to 62468"
this wont even start........ Datz confidence.."
 PO Box 5249
GN
.;-):-D"
just been in bedbut mite go 2 thepub l8tr if uwana mt up?loads a luv Jenxxx.\""
 bt not his girlfrnd... G o o d n i g h t . . .@"
 I'll come up"
 don't miss ur best life for anything... Gud nyt..."
 just as a shop has to give a guarantee on what they sell. B. G."
 But at d end my love compromised me for everything:-(\".. Gud mornin:-)"
 the toughest is acting Happy with all unspoken pain inside..\""
 smoke hella weed\""
\" not \"what i need to do.\""
JUST GOT PAYED2DAY & I HAVBEEN GIVEN A�50 PAY RISE 4MY WORK & HAVEBEEN MADE PRESCHOOLCO-ORDINATOR 2I AM FEELINGOOD LUV\""
 justthought i��d sayhey! how u doin?nearly the endof me wk offdam nevamind!We will have 2Hook up sn if uwant m8? loveJen x.\""
JUST REALLYNEED 2DOCD.

In [104]:
df_1 = df1[df1.iloc[:, 2].isna()]
df_1 = df_1[df_1.iloc[:, 3].isna()]
df_1 = df_1[df_1.iloc[:, 4].isna()]


'''
for indice_fila, fila in df_filtrado.iterrows():
     palabra = fila.iloc[3]  # Acceder al valor en la tercera columna de cada fila
     if not pd.isna(palabra):  # Verificar si el valor no es NaN
        print(palabra)
'''

df_1

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
444,ham,\HEY HEY WERETHE MONKEESPEOPLE SAY WE MONKEYAR...,HOWU DOIN? FOUNDURSELF A JOBYET SAUSAGE?LOVE ...,NaN,NaN
671,spam,SMS. ac sun0819 posts HELLO:\You seem cool,"wanted to say hi. HI!!!\"" Stop? Send STOP to ...",NaN,NaN
710,ham,Height of Confidence: All the Aeronautics prof...,"this wont even start........ Datz confidence..""",NaN,NaN
1127,ham,"Height of \Oh shit....!!\"" situation: A guy th...",".;-):-D""",NaN,NaN
1266,ham,\Hey sorry I didntgive ya a a bellearlier hunny,just been in bedbut mite go 2 thepub l8tr if u...,NaN,NaN
1384,ham,"Storming msg: Wen u lift d phne, u say \HELLO\...","bt not his girlfrnd... G o o d n i g h t . . .@""",NaN,NaN
1428,ham,"Tell you what, if you make a little spreadshee...","I'll come up""",NaN,NaN
1559,ham,Single line with a big meaning::::: \Miss anyt...,don't miss ur best life for anything... Gud n...,NaN,NaN
1637,spam,"0A$NETWORKS allow companies to bill for SMS, s...",just as a shop has to give a guarantee on wha...,NaN,NaN
1669,ham,Very hurting n meaningful lines ever: \I compr...,But at d end my love compromised me for every...,NaN,NaN


In [ ]:
for clase in df1['v1'].unique():
 if not pd.isna(clase):
      # Obtener el primer elemento de cada clase
      primer_elemento = df1[df1['v1'] == clase].iloc[0]
      # Agregarlo al diccionario
      print(primer_elemento)

In [ ]:
print(df1['v1'].value_counts())

Vemos que la cantidad de mensajes del tipo 'ham' aparece mucho más que el tipo 'spam' lo que indica que no estña balanceado. Por tanto, a la hora de sacar conlcusiones tendremos que tener en mente este hecho, ya que podría suceder que no fueran todo lo representativas que qusiéramos.

A continuación normalizaremos el documento eliminando caracteres especiales, pasando a minúsculas y eliminando palabras vacías.

In [48]:
wpt = nltk.WordPunctTokenizer()
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1_normalizado = normalize_corpus(df1['v2'].values)


In [57]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv_matrix = cv.fit_transform(df1_normalizado)
cv_matrix
cv_matrix = cv_matrix.toarray()
cv_matrix
suma_columnas = cv_matrix.sum(axis = 0)
indice_columna_maxima = np.argmax(suma_columnas)

nombres_columnas = cv.get_feature_names_out()

print(nombres_columnas[indice_columna_maxima])
print(np.max(suma_columnas))

call
579


La palabra que más se repite es call y aparece 579 veces.

# 2) Representación como bolsa de palabras

Elige justificadamente una representación de bolsa de palabras y aplícala.
Muestra un ejemplo antes y después de aplicar la representación. Explica los cambios.

# 3) Aplica 3 algoritmos de aprendizaje automático para resolver la tarea

Justifica porqué los has elegido.
Ajusta los modelos respecto a un hiperparámetro que consideres oportuno. Justifica tu elección.
Explica los resultados obtenidos.

# 4) Construye redes neuronales con Keras con distintas maneras de usar word embeddings

Justifica tus decisiones y explica los resultados obtenidos.

# 5) Aplica los modelos construidos a los datos de test y compáralos.

Calcula las métricas de recall, precisión y f1.
Discute cual es el mejor modelo y cual es peor y porqué.